# Payoff d'une iron butterfly sur SPY

TP sur l'iron butterfly : long put aile basse, short put et short call au strike central, long call aile haute.
Objectifs :
1. Fixer un spot de référence via l'historique SPY.
2. Visualiser le plateau central de gain (structure crédit) et les pertes bornées sur les ailes.
3. Relier la combinaison put/call vendus au centre à la forme "papillon" du payoff.

In [ ]:
%pip -q install yfinance matplotlib pandas numpy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use("seaborn-v0_8-darkgrid")

def payoff_call(spot: float, strike: float) -> float:
    return max(spot - strike, 0.0)

def payoff_put(spot: float, strike: float) -> float:
    return max(strike - spot, 0.0)

def payoff_iron_bfly(spot: float, k_put_long: float, k_center: float, k_call_long: float) -> float:
    return (
        payoff_put(spot, k_put_long)
        - payoff_put(spot, k_center)
        - payoff_call(spot, k_center)
        + payoff_call(spot, k_call_long)
    )

def fetch_spy_history(period="1y", interval="1d") -> pd.Series:
    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    return data["Close"]

close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
k_center = spot_ref
k_put_long = spot_ref * 0.95
k_call_long = spot_ref * 1.05
spot_ref, k_put_long, k_center, k_call_long

## Évolution du sous-jacent (SPY)
Clôtures sur un an et repère du spot de référence (dernier close).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()

## Payoff de l'iron butterfly
Plateau de gain autour du strike central, pertes bornées sur les ailes.

In [ ]:
S_grid = np.linspace(spot_ref * 0.5, spot_ref * 1.5, 200)
payoffs = [payoff_iron_bfly(S, k_put_long, k_center, k_call_long) for S in S_grid]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(S_grid, payoffs, color="navy", label="Payoff iron butterfly")
for k, style, label in [
    (k_put_long, ":", f"Put long = {k_put_long:.2f}"),
    (k_center, "--", f"Strike central = {k_center:.2f}"),
    (k_call_long, ":", f"Call long = {k_call_long:.2f}"),
]:
    ax.axvline(k, color="gray", linestyle=style, label=label)
ax.set_xlabel("Spot S")
ax.set_ylabel("Payoff à maturité")
ax.set_title("Iron butterfly sur SPY : payoff vs spot")
ax.legend()
plt.show()